In [1]:
import tensorflow as tf
import os, pickle, sys, time
from collections import defaultdict
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras import Model
from utils.MyRAN56_v1 import ResidualAttentionModel_56 as ran56_v1 
from utils.MyRAN56_v2 import ResidualAttentionModel_56 as ran56_v2
from utils.MyRAN92_v1 import ResidualAttentionModel_92 as ran92_v1
from utils.MyRAN92_v2 import ResidualAttentionModel_92 as ran92_v2
from contextlib import redirect_stdout
from sklearn.metrics import top_k_accuracy_score

In [2]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)
print(gpus)

1 Physical GPUs, 1 Logical GPUs
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [4]:
cifar10 = tf.keras.datasets.cifar10
(X_train, y_train), (X_test, y_test) = cifar10.load_data()
y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)

datagen = tf.keras.preprocessing.image.ImageDataGenerator(rotation_range=20,
                                                          width_shift_range=0.2,
                                                          height_shift_range=0.2,
                                                          horizontal_flip=True,
                                                          validation_split=0.2)

X_train = X_train.astype('float32') / 255
X_test = X_test.astype('float32') / 255

input_shape = X_train.shape[1:]
output_classes = 10

In [5]:
arl_mean_train = X_train.mean()
arl_mean_test = X_test.mean()

nal_mean = np.load("train_mean.npy")
nal_std = np.load("train_std.npy")

In [6]:
model_path = 'models'

# Edit configurations here to autoload models
configs = {
    'ran56_v1': {'model': ran56_v1(input_shape, output_classes, use_bias=True), 'save_path': f'{model_path}/ran56_v1/saved_models/ran56_v1'},
    'ran56_v2_ARL': {'model': ran56_v2(input_shape, output_classes), 'save_path': f'{model_path}/ran56_v2_ARL/ran56_v2'},
    'ran56_v2_nal': {'model': ran56_v2(input_shape, output_classes, learning_type='NAL'), 'save_path': f'{model_path}/ran56_v2_nal/saved_models/ran56_v2'},
    'ran92_v1': {'model': ran92_v1(input_shape, output_classes, use_bias=True), 'save_path': f'{model_path}/ran92_v1/saved_models/ran92_v1_FULL_MODEL_Mon_Dec_20_18_26_00_2021'},
    'ran92_v2_arl': {'model': ran92_v2(input_shape, output_classes), 'save_path': f'{model_path}/ran92_v2_ARL/saved_models/ran92_v2_ARL'},
    'ran92_v2_nal': {'model': ran92_v2(input_shape, output_classes, learning_type='NAL'), 'save_path': f'{model_path}/ran92_v2_nal/saved_models/ran92_v2'},
}



In [7]:
results = {}

# Runs eval for any models it can find and saves to results
for k in configs.keys():
    config = configs[k]
    
    if not os.path.exists(config['save_path']):
        continue 
    print(f"Found model at: {config['save_path']}")

    if '_nal' in k:
        X_test_eval = (X_test - nal_mean) / nal_std
    else:
        X_test_eval = X_test - arl_mean_test

    model = config['model'].return_Model()

    model.compile(tf.keras.optimizers.Adam(), 
                loss=tf.keras.losses.CategoricalCrossentropy(), 
                metrics=['accuracy'])
    
    model.load_weights(config['save_path'])

    preds = model.predict(X_test_eval)
    top1acc = top_k_accuracy_score(np.argmax(y_test, axis=1), preds, k=1)
    top5acc = top_k_accuracy_score(np.argmax(y_test, axis=1), preds, k=5)
    
    top1err = (1.0 - top1acc) * 100
    top5err = (1.0 - top5acc) * 100
    
    results[k] = {'top1err': top1err, 'top5err': top5err}
    


Found model at: models/ran56_v2_ARL/ran56_v2
Found model at: models/ran92_v2_ARL/saved_models/ran92_v2_ARL


In [8]:
results

{'ran56_v2_ARL': {'top1err': 9.430000000000005, 'top5err': 0.5199999999999982},
 'ran92_v2_arl': {'top1err': 12.709999999999999,
  'top5err': 0.5900000000000016}}

In [8]:
top_k_accuracy_score(np.argmax(y_test, axis=1), preds, k=5)

0.9966

In [9]:
top_k_accuracy_score(np.argmax(y_test, axis=1), preds, k=1)

0.9193

In [92]:
a = preds.argsort()[:, -1:]
b = np.argmax(y_test, axis=1)


In [93]:
counter = 0
for f in range(len(b)):
    if b[f] in a[f]:
        counter += 1
counter / len(b)

0.7076